In [ ]:
import cv2
import os , sys
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')
os.chdir("drive/My Drive/Colab Notebooks/recsys_proj//DIEN-tf2")
sys.path.append("drive/My Drive/Colab Notebooks/recsys_proj//DIEN-tf2")

Mounted at /content/drive


In [ ]:
!ls 
# !mkdir data/private/Ad/amazon

amazon_prepro.ipynb  data.py   main.py		models	     run_main.ipynb
config.py	     layer.py  ml_prepro.ipynb	__pycache__  test.py
data		     logs      model.py		README.md    utils.py


In [ ]:
os.path.isdir('data')
# !mkdir ('/data')

True

In [ ]:
import tensorflow as tf
import numpy as np
import csv, os
from tqdm import tqdm
import pandas as pd
import random
import pickle

# root = '/data/private/Ad/amazon/'


In [ ]:
!ls data

cat_voc.pkl		 meta_Electronics.json	     uid_voc.pkl
local_test_splitByUser	 mid_voc.pkl
local_train_splitByUser  reviews_Electronics_5.json


In [ ]:
root = 'data/'
with open(root+'reviews_Electronics_5.json') as fin:
    df = {}
    for i, line in enumerate(fin):
        df[i] = eval(line)
    reviews_df = pd.DataFrame.from_dict(df, orient='index')

In [ ]:
reviews_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [ ]:
with open(root+'reviews.pkl', 'wb') as f:
    pickle.dump(reviews_df, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(root+'meta_Electronics.json') as fin:
    df = {}
    for i, line in enumerate(fin):
        df[i] = eval(line)
    meta_df = pd.DataFrame.from_dict(df, orient='index')

meta_df = meta_df[meta_df['asin'].isin(reviews_df['asin'].unique())]
meta_df = meta_df.reset_index(drop=True)
with open(root+'meta.pkl', 'wb') as f:
    pickle.dump(meta_df, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_df = reviews_df[['reviewerID', 'asin', 'unixReviewTime']]
meta_df = meta_df[['asin', 'categories']]
# only one category...
meta_df['categories'] = meta_df['categories'].map(lambda x: x[-1][-1])

In [ ]:
def build_map(df, col_name):
    key = sorted(df[col_name].unique().tolist())
    m = dict(zip(key, range(len(key))))
    df[col_name] = df[col_name].map(lambda x: m[x])
    return m, key

In [ ]:
asin_map, asin_key = build_map(meta_df, 'asin')
cate_map, cate_key = build_map(meta_df, 'categories')
revi_map, revi_key = build_map(reviews_df, 'reviewerID')

In [ ]:
meta_df.head()

,asin,categories
0,0,738
1,1,157
2,2,571
3,3,707
4,7,799


In [ ]:
user_count, item_count, cate_count, example_count =\
    len(revi_map), len(asin_map), len(cate_map), reviews_df.shape[0]
print('user_count: %d\titem_count: %d\tcate_count: %d\texample_count: %d' %
      (user_count, item_count, cate_count, example_count))

user_count: 192403	item_count: 63001	cate_count: 801	example_count: 1689188


In [ ]:
meta_df = meta_df.sort_values('asin')
meta_df = meta_df.reset_index(drop=True)

In [ ]:
reviews_df['asin'] = reviews_df['asin'].map(lambda x: asin_map[x])
reviews_df = reviews_df.sort_values(['reviewerID', 'unixReviewTime'])
reviews_df = reviews_df.reset_index(drop=True)

In [ ]:
cate_list = [meta_df['categories'][i] for i in range(len(asin_map))]
cate_list = np.array(cate_list, dtype=np.int32)

In [ ]:
reviews_df.head(5)

,reviewerID,asin,unixReviewTime
0,0,13179,1400457600
1,0,17993,1400457600
2,0,28326,1400457600
3,0,29247,1400457600
4,0,62275,1400457600


In [ ]:
with open(root+'remap.pkl', 'wb') as f:
    pickle.dump(reviews_df, f, pickle.HIGHEST_PROTOCOL) # uid, iid
    pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL) # cid of iid line
    pickle.dump((user_count, item_count, cate_count, example_count),
              f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((asin_key, cate_key, revi_key), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
random.seed(1234)

train_set = []
test_set = []
for reviewerID, hist in reviews_df.groupby('reviewerID'):
    pos_list = hist['asin'].tolist()
    neg_list = []
    for _ in range(len(pos_list)):
        neg = pos_list[0]
        while neg in pos_list + neg_list :
            neg = random.randint(0, item_count-1)
        neg_list.append(neg)
        
    for i in range(1, len(pos_list)-1):
        hist = pos_list[:i]
        train_set.append((reviewerID, hist, pos_list[i], 1))
        train_set.append((reviewerID, hist, neg_list[i], 0))
    label = (pos_list[-1], neg_list[-1])
    test_set.append((reviewerID, hist, label))

random.shuffle(train_set)
random.shuffle(test_set)

assert len(test_set) == user_count

with open(root+'dataset.pkl', 'wb') as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((user_count, item_count, cate_count), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# create new data set with time
random.seed(1234)

train_set = []
test_set = []
for reviewerID, hist in reviews_df.groupby('reviewerID'):
    unixReviewTime = hist['unixReviewTime'].mean()
    pos_list = hist['asin'].tolist()
    neg_list = []
    for _ in range(len(pos_list)):
        neg = pos_list[0]
        while neg in pos_list + neg_list :
            neg = random.randint(0, item_count-1)
        neg_list.append(neg)
        
    for i in range(1, len(pos_list)-1):
        
        hist = pos_list[:i]
        train_set.append((reviewerID, hist, pos_list[i],unixReviewTime, 1))
        train_set.append((reviewerID, hist, neg_list[i],unixReviewTime, 0))

    label = (pos_list[-1], neg_list[-1])
    test_set.append((reviewerID, hist, unixReviewTime,label))

random.shuffle(train_set)
random.shuffle(test_set)

# assert len(test_set) == user_count

with open(root+'dataset_with_time.pkl', 'wb') as f:
    pickle.dump(train_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_set, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(cate_list, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump((user_count, item_count, cate_count), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
reviews_df.groupby('reviewerID').head()

,reviewerID,asin,unixReviewTime
0,0,13179,1400457600
1,0,17993,1400457600
2,0,28326,1400457600
3,0,29247,1400457600
4,0,62275,1400457600
...,...,...,...
1689156,192402,24851,1262044800
1689157,192402,29488,1282262400
1689158,192402,36004,1357776000
1689159,192402,37977,1357776000


In [ ]:
i=1
for reviewerID, hist in reviews_df.groupby('reviewerID'):
  i=i+1
  print(hist)
  if i==2:
    break

   reviewerID   asin  unixReviewTime
0           0  13179      1400457600
1           0  17993      1400457600
2           0  28326      1400457600
3           0  29247      1400457600
4           0  62275      1400457600
